In [1]:
import string
import re 
import csv
import collections

import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import islice

import nltk
from nltk.tokenize import sent_tokenize

from datasets import load_metric, load_dataset

### Load Summaries

Transformers
- BART: sshleiferdistilbart-xsum-12-1
- Pegasus: sshleiferdistill-pegasus-xsum-16-4
- GPT-5: t5-small
- BigBird: googlebigbird-pegasus-large-bigpatent

Baselines

- Lead 3

Trained Transformers

- T5
- Pegasus
- BART

In [2]:
#Transformers
bart    = pd.read_csv('data/output_sshleiferdistilbart-xsum-12-1.csv')
pegasus = pd.read_csv('data/output_sshleiferdistill-pegasus-xsum-16-4.csv')
t5      = pd.read_csv('data/output_t5-small.csv')
bird    = pd.read_csv('data/output_googlebigbird-pegasus-large-bigpatent.csv')

#Baselines
lead_three = pd.read_csv('data/lead3_patents.csv')

#Trained
t5_trained = pd.read_csv('data/output_trained_modelst5_trained.csv')
pegasus_trained = pd.read_csv('data/output_trained_modelspegasus_trained.csv')
bart_trained = pd.read_csv('data/output_trained_modelsbart_trained.csv')

# #Adjust columns
bart = bart[['ground_truth', 'summaries']]
bart.columns = ['ground_truth', 'bart']

pegasus = pegasus[['ground_truth', 'summaries']]
pegasus.columns = ['ground_truth', 'pegasus']

t5 = t5[['ground_truth', 'summaries']]
t5.columns = ['ground_truth', 't5']

bird = bird[['ground_truth','summaries']]
bird.columns = ['ground_truth', 'bird']

t5_trained = t5_trained[['ground_truth', 'summaries']]
t5_trained.columns = ['ground_truth', 't5_trained']

pegasus_trained = pegasus_trained[['ground_truth', 'summaries']]
pegasus_trained.columns = ['ground_truth', 'pegasus_trained']

bart_trained = bart_trained[['ground_truth', 'summaries']]
bart_trained.columns = ['ground_truth', 'bart_trained']

In [3]:
bart = bart.drop_duplicates(subset='ground_truth', keep='first')
pegasus = pegasus.drop_duplicates(subset='ground_truth', keep='first')
t5 = t5.drop_duplicates(subset='ground_truth', keep='first')

lead_three = lead_three.drop_duplicates(subset='ground_truth', keep='first')

t5_trained = t5_trained.drop_duplicates(subset='ground_truth', keep='first')
pegasus_trained = pegasus_trained.drop_duplicates(subset='ground_truth', keep='first')
bart_trained = bart_trained.drop_duplicates(subset='ground_truth', keep='first')

print(len(pegasus))
print(len(bart))
print(len(t5))
print(len(lead_three))
print(len(t5_trained))
print(len(pegasus_trained))
print(len(bart_trained))

13948
13948
13948
13948
13948
13948
13948


In [4]:
temp  = bart.merge(pegasus, how='inner', on='ground_truth')
temp2 = temp.merge(t5, how='inner', on='ground_truth')
temp3 = temp2.merge(lead_three, how='inner', on='ground_truth')
temp4 = temp3.merge(bird, how='inner', on='ground_truth')
temp5 = temp4.merge(t5_trained, how='inner', on='ground_truth')
temp6 = temp5.merge(pegasus_trained, how='inner', on='ground_truth')
merged = temp6.merge(bart_trained, how='inner', on='ground_truth')

merged

,ground_truth,bart,pegasus,t5,lead3,bird,t5_trained,pegasus_trained,bart_trained
0,a technique is described for completely and po...,The first picture of a home laundry dryer has...,This is an illustration of the type of machine...,fig1 is an exploded rear perspective view of a...,fig1 is an exploded rear perspective view of a...,"A wiring assembly for home laundry dryers, etc...",of a home laundry dryer which can be wired by ...,a method and apparatus for wiring a machine wh...,a method and apparatus for assembling a machin...
1,a method and system for pseudorandom noise pha...,pn has been presented with a dtmb system for ...,A method of detecting the pn phase in the freq...,a method that estimates the pn phase in the fr...,in a presently preferred method of the present...,A method for estimating the pn phase in the fr...,a method for detecting the pn phases of the sy...,a method for estimating the pn phase in a freq...,a method for estimating a pn phase in a freque...
2,a semiconductor device and a method for produc...,The scope of a semiconductor device that can ...,The following is the invention of a semiconduc...,is based on the consideration that the interfa...,"in the following detailed description , refere...",In a semiconductor device having a circuit car...,a semiconductor device includes a semiconducto...,an adhesion - promoting layer is provided betw...,the present invention relates to a semiconduct...
3,the present invention relates to a dual damasc...,The development of the dual damascene has rev...,Fig3 a to 3i are cross-sectional views showing...,"u. s. pat. no. 5, 3",fig3 a to 3i are cross - sectional views showi...,A method for manufacturing the dual damascene ...,for manufacturing a dual damascene. the dual d...,a method for manufacturing a dual damascene in...,a method for manufacturing the dual damascene ...
4,"an internet broadcast system , method and appa...",The invention of a digital platform that allo...,The following is the invention of an internet ...,in accordance with a preferred embodiment of t...,a platform and production process ( hereinafte...,A platform and production process that offers ...,a platform and production process that offers ...,"a platform and method for providing a rich, vi...",a platform and method for producing a multi - ...
...,...,...,...,...,...,...,...,...,...
14274,"a data reading apparatus , includes an apparat...",A device that uses a memory card that can be ...,A data reading apparatus is used to read a rec...,"of the present invention, as a data reading ap...","hereinafter , an embodiment of the present inv...",A data reading apparatus includes a specific i...,a data reading apparatus used to read a record...,the present invention relates to a data readin...,a data reading apparatus includes a reading me...
14275,an integrated circuit for supplying power incl...,"The invention of a communication apparatus, w...",The following is the invention of a power supp...,", embodiments of the present invention will be...","in the following , embodiments of the present ...",A power supply ic according to the present inv...,a communication apparatus includes a power sup...,a power supply circuit unit having a plurality...,a power supply ic includes : a power supply ci...
14276,a method and apparatus for allowing the user o...,The detailed details of a power generator tha...,The disclosure of a power generation system 10...,"is now directed to the drawings, where like or...","reference is now directed to the drawings , wh...",A power generation system includes a power gen...,a power generation system includes a power sup...,a power generation system includes a power gen...,a power generation system includes a power gen...
14277,a signal processing circuit outputs a digital ...,A cmos ( cmos) in order to read images from 1...,a cmos image circuit according to an embodimen...,is described below with reference to fig1. a s...,a cmos image circuit according to an embodimen...,A sensor

In [5]:
merged.isnull().any()

ground_truth       False
bart               False
pegasus            False
t5                 False
lead3              False
bird               False
t5_trained         False
pegasus_trained    False
bart_trained       False
dtype: bool

In [6]:
merged.isnull().sum()

ground_truth       0
bart               0
pegasus            0
t5                 0
lead3              0
bird               0
t5_trained         0
pegasus_trained    0
bart_trained       0
dtype: int64

### Cleaning

Cleaning function replaces white spaces and newlines

In [5]:
def clean(s):
    a = s.translate(str.maketrans(' ', ' ', string.punctuation))
    return re.sub('\s+',' ', a).replace('\n', '').strip()

In [6]:
for c in list(merged.columns):
    merged[c] = [clean(series) for series in list(merged[c])]

In [7]:
merged_dict = merged.to_dict('index')

### Example

Below is a series of summaries for a given article

In [11]:
merged_dict[4]

{'ground_truth': 'an internet broadcast system method and apparatus provides a presentation tool that offers program content to viewers in a rich interactive and exciting way while still offering the benefits of cost effectiveness convenience and which meets or exceeds the experience of live in person programming at least two synchronized images are presented in multiple windows and an editable image selected from one of the at least two synchronized images wherein the other of the at least two synchronized image adjust to remain in sync with the edited image a production panel includes an administrative section and a viewer section the administrative section includes set of production tools which allow an administrator to create and modify programming that a viewer observes and engages with through the viewer section via a remote digital viewing device information flows between the production panel and viewing devices via the cloud based software',
 'bart': 'The invention of a digital

### ROUGE Batching

In [11]:
def chunks(dictionary, batch_size=10):
    """Yield successive batch-sized chunks from dictionary."""
    it = iter(dictionary)   
    for i in range(0, len(dictionary), batch_size):
        yield {k: dictionary[k] for k in islice(it, batch_size)}
    
chunk_list = [c for c in chunks(merged_dict)]
print(f"Number of batches: {len(chunk_list)}")

Number of batches: 1428


In [12]:
results = {}

rouge_metric = load_metric("rouge")

#ROUGE Metric Batching
for summarizer in list(merged.columns)[1:]:
    for chunk in tqdm(chunk_list):
        refs  = [i['ground_truth'] for i in list(chunk.values())]
        preds = [i[summarizer] for i in list(chunk.values())]
        rouge_metric.add_batch(predictions=preds, references=refs)

    scores = rouge_metric.compute()
    results[summarizer] = [
        np.round(scores['rouge1'].mid.fmeasure * 100, 1), 
        np.round(scores['rougeL'].mid.fmeasure * 100, 1)]

100%|██████████| 1428/1428 [00:00<00:00, 2545.19it/s]


### BLEU Batching

In [13]:
bleu_metric  = load_metric('bleu')

#BLEU Metric Batching
for summarizer in list(merged.columns)[1:]:

    for chunk in tqdm(chunk_list):
        refs  = [i['ground_truth'].split() for i in list(chunk.values())]        
        refs  = list(map(lambda item: [item], refs))
        preds = [i[summarizer].split() for i in list(chunk.values())]

        bleu_metric.add_batch(predictions=preds, references=refs)

    bleus = bleu_metric.compute()   
    results[summarizer].extend([
        np.round(bleus['bleu'] * 100, 3),
#         np.round(bleus['precisions'][0] * 100, 1),
#         np.round(bleus['brevity_penalty'], 3),
        np.round(bleus['length_ratio'], 3)])

100%|██████████| 1428/1428 [00:05<00:00, 258.67it/s]


### Perplexity

In [14]:
### Unigram and Perplexity
def unigram(tokenized_corpus):
    model = collections.defaultdict(lambda: 0.01)
    for f in tqdm(tokenized_corpus):
        try: model[f] += 1
        except KeyError: 
            model[f] = 1
            continue
    N = float(sum(model.values()))
    for word in model:
        model[word] = model[word]/N
    return model

def perplexity(string, model):
    tokens = string.split()
    perplexity = 1
    N = 0
    for w in tokens:
        N += 1
        perplexity = perplexity * (1 / model[w])
    perplexity = pow(perplexity, 1 / float(N))
    return perplexity

In [15]:
DATA_CACHE_PATH = 'cached_data'

### EDIT THIS LINE FOR YOUR DATASET
dataset = load_dataset('big_patent', 'h', cache_dir=DATA_CACHE_PATH)

Reusing dataset big_patent (cached_data/big_patent/h/1.0.0/bdefa7c0b39fba8bba1c6331b70b738e30d63c8ad4567f983ce315a5fef6131c)


  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
#We need all of the words from both articles and their abstracts to form a vocabulary
#for the unigram model

#You may need to adjust the column names (e.g. description)
train_texts = list(dataset['train']['description']) + list(dataset['train']['abstract'])
test_texts = list(dataset['test']['description']) + list(dataset['test']['abstract'])
all_texts = train_texts + test_texts
cleaned = [clean(s) for s in tqdm(all_texts)]

token_list = [s.split() for s in tqdm(cleaned)]
flattened = [a for sublist in token_list for a in sublist]

100%|██████████| 542596/542596 [09:33<00:00, 946.90it/s]  


In [17]:
unigram_model = unigram(flattened)

100%|██████████| 880589309/880589309 [04:51<00:00, 3026048.12it/s]


In [18]:
perplexity_results = {}

for summarizer in list(merged.columns)[1:]:
    column = list(merged[summarizer])
    perplexity_results[summarizer] = [perplexity(i, unigram_model) for i in column]

In [19]:
print(np.mean(perplexity_results['bart']))
print(np.mean(perplexity_results['pegasus']))
print(np.mean(perplexity_results['t5']))
print(np.mean(perplexity_results['bird']))
print(np.mean(perplexity_results['lead3']))
print(np.mean(perplexity_results['t5_trained']))
print(np.mean(perplexity_results['pegasus_trained']))
print(np.mean(perplexity_results['bart_trained']))

print("")

print(np.median(perplexity_results['bart']))
print(np.median(perplexity_results['pegasus']))
print(np.median(perplexity_results['t5']))
print(np.median(perplexity_results['bird']))
print(np.median(perplexity_results['lead3']))
print(np.median(perplexity_results['t5_trained']))
print(np.median(perplexity_results['pegasus_trained']))
print(np.median(perplexity_results['bart_trained']))

458.4523960494196
691.161122763715
3674.3997214110864
inf
inf
1259.9834948945834
1248.4044247277918
642.7847531943219

390.9595093362846
527.793767792649
680.4037603280391
863.2131693695587
779.2523326124232
816.2375853446887
740.4797159366858
562.4215914620626


In [20]:
for k, v in perplexity_results.items():
    results[k].append(int(np.round(np.median(perplexity_results[k]))))

In [21]:
labels = [
    'rouge1', 'rougeL',
    'bleu', 
#     'precisions', 'brevity_penalty', 
    'length_ratio',
    'perplexity'
]

df = pd.DataFrame.from_dict(results)
df = df.T
df.columns = labels
df

,rouge1,rougeL,bleu,length_ratio,perplexity
bart,20.2,14.6,0.267,0.252,391.0
pegasus,24.1,16.7,1.284,0.351,528.0
t5,10.0,8.3,0.003,0.112,680.0
lead3,28.6,18.0,4.616,0.728,779.0
bird,31.8,22.2,5.142,0.595,863.0
t5_trained,13.2,11.2,0.006,0.110,816.0
pegasus_trained,28.5,21.6,3.297,0.448,740.0
bart_trained,35.1,24.0,4.405,0.459,562.0
